<a href="https://colab.research.google.com/github/monda00/horse-race-notebook/blob/master/horse_race_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# netkeibaでスクレイピングしたデータのEDA

- 準備
- ライブラリ・データ読み込み
- データの概観・分析
- 学習用データ作成

# 準備

## ドメイン知識

- あがり
  - レースや調教の終盤の走破タイム
- 複勝
  - ３着までに入る馬を当てる馬券
  - ５頭以上の場合のみ
  - ７頭以下の場合は２着まで

## データ

- 2019年から2020年までのレースデータ

## 特徴量として利用できるデータ

以下のデータはレース後にしか取得できないため、特徴量として利用しない

- field_condition
- time
- agari
- rank
- horse_weight

## 目的変数

### 何を予測するか

- ３着以内に入る確率が最も高い馬
  - 複勝を予測する
    - ８頭以上のレースに限られる

### どう予測させるか

- ３着以内のデータを追加
  - 1~3着のデータが1、それ以外は0
  - 最も確率の高いラベルがついているのを出力とする
- ランキング学習で１位の馬
  - ランキング学習について調査する必要あり

# ライブラリ・データ読み込み

In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA_PATH = '/content/drive/My Drive/data/horse-race/'

In [3]:
race_df = pd.read_csv(DATA_PATH + 'race.csv')
horse_df = pd.read_csv(DATA_PATH + 'horse.csv')

In [4]:
race_df.head()

,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather
0,右,2020/6/22,1400,不,ダ,タイタン賞競走,大井,202044062212,12R,20:50,雨
1,右,2020/6/22,1800,不,ダ,ポートサイド賞競走,大井,202044062211,11R,20:10,雨
2,右,2020/6/22,1200,不,ダ,C1九 十11,大井,202044062210,10R,19:30,雨
3,右,2020/6/22,1400,不,ダ,C1九 十11,大井,202044062209,9R,18:55,雨
4,右,2020/6/22,1600,不,ダ,C1九 十11,大井,202044062208,8R,18:20,雨


In [5]:
horse_df.head()

,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win
0,38.1,牡3,7.0,13,511(-5),森泰斗,ワイルドホース,2.0,2020/6/22,202044062212,タイタン賞競走,1,1:28.5,54.0,3.7
1,38.1,牝5,6.0,11,475(-2),山崎誠士,クインズオライリー,12.0,2020/6/22,202044062212,タイタン賞競走,2,1:28.7,54.0,81.4
2,38.6,セ6,3.0,4,469(+2),西啓太,メダーリアフレイム,4.0,2020/6/22,202044062212,タイタン賞競走,3,1:28.8,56.0,8.6
3,37.9,牡4,8.0,14,526(+6),藤本現暉,ゴールドプリンス,8.0,2020/6/22,202044062212,タイタン賞競走,4,1:28.8,56.0,34.4
4,38.5,牝5,4.0,7,475(+6),藤田凌,イグレット,9.0,2020/6/22,202044062212,タイタン賞競走,5,1:28.9,54.0,34.8


# データの概観・分析

In [6]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27532 entries, 0 to 27531
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   clockwise        27532 non-null  object
 1   date             27532 non-null  object
 2   distance         27532 non-null  int64 
 3   field_condition  25100 non-null  object
 4   field_type       25100 non-null  object
 5   name             27532 non-null  object
 6   place            27532 non-null  object
 7   race_id          27532 non-null  object
 8   race_round       27532 non-null  object
 9   start_time       26755 non-null  object
 10  weather          26690 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.3+ MB


In [7]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293006 entries, 0 to 293005
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   agari         264082 non-null  float64
 1   age           293005 non-null  object 
 2   frame_number  290217 non-null  float64
 3   horse_number  293006 non-null  int64  
 4   horse_weight  293006 non-null  object 
 5   jockey        291874 non-null  object 
 6   name          293003 non-null  object 
 7   popular       289280 non-null  float64
 8   race_date     293006 non-null  object 
 9   race_id       293006 non-null  object 
 10  race_name     293006 non-null  object 
 11  rank          293006 non-null  int64  
 12  time          286445 non-null  object 
 13  weight        293004 non-null  float64
 14  win           293006 non-null  object 
dtypes: float64(4), int64(2), object(9)
memory usage: 33.5+ MB


## データ数

In [8]:
print('race:', race_df.shape[0])
print('horse:', horse_df.shape[0])

race: 27532
horse: 293006


## カラム

In [9]:
race_df.columns

Index(['clockwise', 'date', 'distance', 'field_condition', 'field_type',
       'name', 'place', 'race_id', 'race_round', 'start_time', 'weather'],
      dtype='object')

In [10]:
horse_df.columns

Index(['agari', 'age', 'frame_number', 'horse_number', 'horse_weight',
       'jockey', 'name', 'popular', 'race_date', 'race_id', 'race_name',
       'rank', 'time', 'weight', 'win'],
      dtype='object')

## 欠損値の確認

### レース

In [11]:
race_df.isnull().sum()

clockwise             0
date                  0
distance              0
field_condition    2432
field_type         2432
name                  0
place                 0
race_id               0
race_round            0
start_time          777
weather             842
dtype: int64

レース

- field_conditionとfield_typeがないのは障害レースの可能性あり
  - 同じデータ
  - 障害レースにはfield conditionとfield_typeがない
  - 学習データとしては削除しても良さそう
- start_timeがないのはレースが中止？
  - 海外のレースがほとんど
  - 学習データとしては削除しても良さそう
- weatherがないのは？？
  - 海外のレースでない物がある（start_timeがないのと同じ）
  - 日本のレースはいくつか欠損している
    - 開催されていないレースなのでデータ削除

field_conditionとfield_typeが欠損しているデータ

In [12]:
(race_df[race_df['field_condition'].isnull()]['race_id'] == race_df[race_df['field_type'].isnull()]['race_id']).value_counts()

True    2432
Name: race_id, dtype: int64

In [13]:
race_df[race_df['field_condition'].isnull()].head(10)

,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather
24,00,2020/6/22,200,NaN,NaN,B3ー6,帯広(ば),202065062211,11R,20:45,曇
25,00,2020/6/22,200,NaN,NaN,第11回ゴールドトロ,帯広(ば),202065062210,10R,20:05,曇
26,00,2020/6/22,200,NaN,NaN,B4ー3,帯広(ば),202065062209,9R,19:25,曇
27,00,2020/6/22,200,NaN,NaN,B4ー7,帯広(ば),202065062208,8R,18:45,曇
28,00,2020/6/22,200,NaN,NaN,B2ー3,帯広(ば),202065062207,7R,18:10,曇
29,00,2020/6/22,200,NaN,NaN,C1ー4,帯広(ば),202065062206,6R,17:35,曇
30,00,2020/6/22,200,NaN,NaN,2歳 未受賞,帯広(ば),202065062205,5R,17:00,曇
31,00,2020/6/22,200,NaN,NaN,2歳 新馬,帯広(ば),202065062204,4R,16:25,曇
32,00,2020/6/22,200,NaN,NaN,2歳 新馬,帯広(ば),202065062203,3R,15:50,曇
33,00,2020/6/22,200,NaN,NaN,2歳 新馬,帯広(ば),202065062202,2R,15:15,曇


In [14]:
race_df[race_df['field_condition'].isnull()]['place'].value_counts()

帯広(ば)    2432
Name: place, dtype: int64

帯広で開催されるのは、短距離（200m）の特殊なレースなので、データを削除する。

In [15]:
race_df = race_df.dropna(subset=['field_condition'])

In [16]:
race_df.isnull().sum()

clockwise            0
date                 0
distance             0
field_condition      0
field_type           0
name                 0
place                0
race_id              0
race_round           0
start_time         777
weather            842
dtype: int64

In [17]:
race_df[race_df['start_time'].isnull()]['place'].value_counts()

アメリカ        106
イギリス        101
シャティン        72
オーストラリア      72
メイダン         64
ランドウ         44
アイルランド       43
サンタアニタ       40
ロンシャン        31
アスコット        29
コーフィー        28
フランス         26
フレミントン       25
ベルモント        13
シャンティイ       12
チャーチル        12
アラブ首長国連邦     11
ドイツ          10
レパーズタウン      10
ニュージーランド      8
ドーヴィル         7
カナダ           4
ムーニーバレー       3
カタール          3
香港            2
シンガポール        1
Name: place, dtype: int64

レースに出ている馬のデータはどうなっているか

In [18]:
horse_df[horse_df['race_id']=='2020A0a00404']

,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win
3068,NaN,セ4,NaN,6,計不,ドイル,Lord North,3.0,2020/6/17,2020A0a00404,プリンスオブウェール,1,2:05.63,57.0,---
3069,NaN,セ6,NaN,3,計不,ビュイッ,バーニーロイ,4.0,2020/6/17,2020A0a00404,プリンスオブウェール,2,NaN,57.0,---
3070,NaN,牡4,NaN,5,計不,ムーア,Japan,1.0,2020/6/17,2020A0a00404,プリンスオブウェール,3,NaN,57.0,---
3082,NaN,牡4,NaN,2,計不,デソウサ,Bangkok,7.0,2020/6/17,2020A0a00404,プリンスオブウェール,4,NaN,57.0,---
3083,NaN,牝4,NaN,7,計不,デットー,Mehdaayih,6.0,2020/6/17,2020A0a00404,プリンスオブウェール,5,NaN,55.5,---
3084,NaN,牡4,NaN,4,計不,ワトソン,Headman,5.0,2020/6/17,2020A0a00404,プリンスオブウェール,6,NaN,57.0,---


海外のレースなのでデータを削除

In [19]:
race_df = race_df.dropna(subset=['start_time'])

In [20]:
race_df.isnull().sum()

clockwise           0
date                0
distance            0
field_condition     0
field_type          0
name                0
place               0
race_id             0
race_round          0
start_time          0
weather            71
dtype: int64

In [21]:
race_df[race_df['weather'].isnull()].tail(10)

,clockwise,date,distance,field_condition,field_type,name,place,race_id,race_round,start_time,weather
19174,右,2019/6/18,1600,ダ1600m(右),ダ,穂高特別,笠松,201947061810,10R,16:55,NaN
19175,右,2019/6/18,1400,ダ1400m(右),ダ,白馬岳賞,笠松,201947061809,9R,16:20,NaN
19176,右,2019/6/18,1400,ダ1400m(右),ダ,立山賞,笠松,201947061808,8R,15:45,NaN
19177,右,2019/6/18,1400,ダ1400m(右),ダ,サラ系B4組,笠松,201947061807,7R,15:10,NaN
19178,右,2019/6/18,1400,ダ1400m(右),ダ,サラ系C16組,笠松,201947061806,6R,14:35,NaN
19179,右,2019/6/18,1400,ダ1400m(右),ダ,満仲一也ホールインワ,笠松,201947061805,5R,14:00,NaN
19180,右,2019/6/18,1400,ダ1400m(右),ダ,サラ系C18組,笠松,201947061804,4R,13:25,NaN
19181,右,2019/6/18,1400,ダ1400m(右),ダ,サラ系C20組,笠松,201947061803,3R,12:50,NaN
19182,右,2019/6/18,1400,ダ1400m(右),ダ,サラ系C21組,笠松,201947061802,2R,12:15,NaN
19183,右,2019/6/18,800,ダ800m(右),ダ,サラ系C19組,笠松,201947061801,1R,11:45,NaN


In [22]:
horse_df[horse_df['race_id']=='201947061809']

,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win
203930,NaN,セ7,1.0,1,計不,松本剛志,ディーエスノーブル,NaN,2019/6/18,201947061809,白馬岳賞,1,NaN,56.0,---
203931,NaN,牡4,2.0,2,計不,藤原幹生,ハーリーバーリー,NaN,2019/6/18,201947061809,白馬岳賞,2,NaN,56.0,---
203935,NaN,牡4,3.0,3,計不,向山牧,ヒルノコルドバ,NaN,2019/6/18,201947061809,白馬岳賞,3,NaN,56.0,---
203936,NaN,牝4,4.0,4,計不,佐藤友則,バレンティーノ,NaN,2019/6/18,201947061809,白馬岳賞,4,NaN,54.0,---
203937,NaN,牝5,5.0,5,計不,東川慎,ヤマニンミモレット,NaN,2019/6/18,201947061809,白馬岳賞,5,NaN,51.0,---
203944,NaN,牝4,6.0,6,計不,宮下瞳,キラキラオーラ,NaN,2019/6/18,201947061809,白馬岳賞,6,NaN,52.0,---
203945,NaN,セ7,7.0,7,計不,高木健,ブラックバード,NaN,2019/6/18,201947061809,白馬岳賞,7,NaN,56.0,---
203946,NaN,牡4,8.0,8,計不,吉井友彦,ヤマニンフレッチェ,NaN,2019/6/18,201947061809,白馬岳賞,8,NaN,56.0,---
203947,NaN,牡8,8.0,9,計不,山下雅之,ジュガンティーヤ,NaN,2019/6/18,201947061809,白馬岳賞,9,NaN,56.0,---


In [23]:
race_df[race_df['weather'].isnull()]['date'].value_counts()

2019/8/15     12
2019/7/21     12
2019/11/18    12
2019/6/20     11
2019/6/19     10
2019/6/18     10
2019/10/19     2
2019/10/6      1
2019/10/26     1
Name: date, dtype: int64

開催されていないレースのようなので削除

In [24]:
race_df = race_df.dropna(subset=['weather'])

In [25]:
race_df.isnull().sum()

clockwise          0
date               0
distance           0
field_condition    0
field_type         0
name               0
place              0
race_id            0
race_round         0
start_time         0
weather            0
dtype: int64

### 馬

まずは、削除されたレースは削除する

In [26]:
race_id_list = race_df['race_id'].values
horse_race_id_list = horse_df['race_id'].unique()
drop_race_id = []
for race_id in horse_race_id_list:
  if not race_id in race_id_list:
    drop_race_id.append(race_id)

In [27]:
for race_id in drop_race_id:
  horse_df = horse_df[horse_df['race_id'] != race_id]

In [28]:
horse_df.isnull().sum()

agari           3393
age                0
frame_number      69
horse_number       0
horse_weight       0
jockey             0
name               0
popular         2371
race_date          0
race_id            0
race_name          0
rank               0
time            3376
weight             0
win                0
dtype: int64

馬

- agariはデータがないのが多い（障害レース？）
  - agariがない馬はtimeもない
  - timeがないので削除
  - timeが欠損していないがagariが欠損しているのはデータがほとんど欠損しているレースのため削除
- frame_number、jockey、popularは？？
- timeがないのはレースに出場しなかった？
  - レースに関係しないので削除
  - 失格や出ていないなど

In [29]:
horse_df[horse_df['agari'].isnull() ]['time'].isnull().sum()

3376

timeがないのはレースに関係していないので削除

In [30]:
horse_df = horse_df.dropna(subset=['time'])

In [31]:
horse_df.isnull().sum()

agari           17
age              0
frame_number     6
horse_number     0
horse_weight     0
jockey           0
name             0
popular          0
race_date        0
race_id          0
race_name        0
rank             0
time             0
weight           0
win              0
dtype: int64

In [32]:
horse_df[horse_df['agari'].isnull()]

,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win
88930,NaN,牡4,7.0,8,536(+6),矢野貴之,キングバイデン,2.0,2020/1/8,202043010812,ふなばしセレクション,1,1:43.0,56.5,2.8
88940,NaN,牝4,3.0,3,452(+9),張田昂,エスケイエンジェル,4.0,2020/1/8,202043010812,ふなばしセレクション,2,1:43.2,54.5,10.8
88941,NaN,牡8,1.0,1,483(+8),田中力,レジェンドセプター,7.0,2020/1/8,202043010812,ふなばしセレクション,3,1:43.3,56.5,35.0
88947,NaN,セ6,8.0,10,504(+2),本田正重,メイドインドリーム,1.0,2020/1/8,202043010812,ふなばしセレクション,4,1:43.3,56.5,2.7
88948,NaN,牝5,4.0,4,490(0),澤田龍哉,タオルデスカーフ,8.0,2020/1/8,202043010812,ふなばしセレクション,5,1:43.4,54.5,73.3
88953,NaN,牡7,7.0,9,497(+1),森泰斗,キングズミッション,3.0,2020/1/8,202043010812,ふなばしセレクション,6,1:44.3,56.5,3.9
88954,NaN,牡5,8.0,11,418(+8),笠野雄大,エスティブレイド,9.0,2020/1/8,202043010812,ふなばしセレクション,7,1:44.6,56.5,110.7
88960,NaN,牝5,5.0,5,487(+1),山口達弥,リンノストーン,6.0,2020/1/8,202043010812,ふなばしセレクション,8,1:44.9,54.5,21.2
88961,NaN,牡4,6.0,6,460(-4),野澤憲彦,ヤマノプレミアム,10.0,2020/1/8,202043010812,ふなばしセレクション,9,1:45.3,56.5,113.4
88969,NaN,牡6,2.0,2,540(-4),岡村健司,エイシンユニコーン,11.0,2020/1/8,202043010812,ふなばしセレクション,10,1:46.5,56.5,180.9


余ったagariはtimeがほとんどないレース、もしくはふなばしセレクション。
ふなばしセレクションは全ての馬がagariのデータがないため、削除。

In [33]:
horse_df = horse_df.dropna(subset=['agari'])

In [34]:
horse_df.isnull().sum()

agari           0
age             0
frame_number    0
horse_number    0
horse_weight    0
jockey          0
name            0
popular         0
race_date       0
race_id         0
race_name       0
rank            0
time            0
weight          0
win             0
dtype: int64

馬データがないレースデータを削除する。

In [35]:
horse_race_id_list = horse_df['race_id'].unique()
type(horse_race_id_list)

numpy.ndarray

In [36]:
race_id_list = race_df['race_id'].values
horse_race_id_list = horse_df['race_id'].unique()
print(race_id_list.shape)
print(horse_race_id_list.shape)
drop_race_id = []
for race_id in race_id_list:
  if not race_id in horse_race_id_list:
    drop_race_id.append(race_id)

(24252,)
(24221,)


In [37]:
for race_id in drop_race_id:
  race_df = race_df[race_df['race_id'] != race_id]

## ユニークな値の数

In [38]:
for col, values in race_df.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))

clockwise: 11
date: 539
distance: 54
field_condition: 4
field_type: 3
name: 7426
place: 24
race_id: 24221
race_round: 12
start_time: 149
weather: 6


In [39]:
for col, values in horse_df.iteritems():
    num_uniques = values.nunique()
    print ('{name}: {num_unique}'.format(name=col, num_unique=num_uniques))

agari: 264
age: 40
frame_number: 8
horse_number: 18
horse_weight: 8203
jockey: 450
name: 23930
popular: 18
race_date: 539
race_id: 24221
race_name: 7426
rank: 18
time: 1836
weight: 26
win: 5403


- race_idの数がhorse_dfが小さい（２つ足りない）


## データの保存

一旦、保存する。

In [40]:
race_df.to_csv(DATA_PATH + 'race_ex.csv')
horse_df.to_csv(DATA_PATH + 'horse_ex.csv')

## 7頭以下のレース

7頭以下のレースは複勝が２着以内に、５頭未満は複勝がないため、学習データとして使わない。

In [41]:
horse_df['race_id'].value_counts()

202010010108    18
201905050603    18
201906050103    18
202010010809    18
201908040106    18
                ..
202055050401     4
201948062802     4
201945020101     4
201947070506     3
201935071404     3
Name: race_id, Length: 24221, dtype: int64

In [42]:
drop_races_s = horse_df['race_id'].value_counts() <= 7
drop_race_id = drop_races_s[drop_races_s == True].index

In [43]:
for race_id in drop_race_id:
  race_df = race_df[race_df['race_id'] != race_id]

In [44]:
for race_id in drop_race_id:
  horse_df = horse_df[horse_df['race_id'] != race_id]

In [45]:
horse_df['race_id'].value_counts()

201904021110    18
201904030308    18
202010010604    18
201905050601    18
201905050411    18
                ..
201946081805     8
201950083004     8
201955090103     8
201905020506     8
201935052004     8
Name: race_id, Length: 22630, dtype: int64

## 型の確認

- ageは性別？と年齢が合わせっているためobject
- race_dateは/区切りのためobject
- horse_weightは使わないデータのため無視
  - 体重と増減が合わせっているためobject
- timeは使わないため無視
  - 時間表記のためobject
- winは単勝のため数字のはず

winだけ確認する。

In [46]:
horse_df.dtypes

agari           float64
age              object
frame_number    float64
horse_number      int64
horse_weight     object
jockey           object
name             object
popular         float64
race_date        object
race_id          object
race_name        object
rank              int64
time             object
weight          float64
win              object
dtype: object

In [47]:
race_df.dtypes

clockwise          object
date               object
distance            int64
field_condition    object
field_type         object
name               object
place              object
race_id            object
race_round         object
start_time         object
weather            object
dtype: object

winをfloat64に変換する。

In [48]:
horse_df['win'].value_counts()

1.8      1311
1.7      1298
3.0      1271
1.9      1255
3.1      1252
         ... 
672.4       1
620.3       1
499.8       1
403.3       1
402.3       1
Name: win, Length: 5403, dtype: int64

In [49]:
horse_df = horse_df.astype({'win': 'float64'})

In [50]:
horse_df.dtypes

agari           float64
age              object
frame_number    float64
horse_number      int64
horse_weight     object
jockey           object
name             object
popular         float64
race_date        object
race_id          object
race_name        object
rank              int64
time             object
weight          float64
win             float64
dtype: object

# 学習用データの作成

## ３着以内の馬を予測する場合のデータ作成

３着以内の特徴量を追加する。

３着以内なら1、そうでなければ0



In [51]:
horse_df['show'] = 0

In [52]:
horse_df.loc[horse_df['rank'] <= 3, 'show'] = 1

In [53]:
horse_df.head()

,agari,age,frame_number,horse_number,horse_weight,jockey,name,popular,race_date,race_id,race_name,rank,time,weight,win,show
0,38.1,牡3,7.0,13,511(-5),森泰斗,ワイルドホース,2.0,2020/6/22,202044062212,タイタン賞競走,1,1:28.5,54.0,3.7,1
1,38.1,牝5,6.0,11,475(-2),山崎誠士,クインズオライリー,12.0,2020/6/22,202044062212,タイタン賞競走,2,1:28.7,54.0,81.4,1
2,38.6,セ6,3.0,4,469(+2),西啓太,メダーリアフレイム,4.0,2020/6/22,202044062212,タイタン賞競走,3,1:28.8,56.0,8.6,1
3,37.9,牡4,8.0,14,526(+6),藤本現暉,ゴールドプリンス,8.0,2020/6/22,202044062212,タイタン賞競走,4,1:28.8,56.0,34.4,0
4,38.5,牝5,4.0,7,475(+6),藤田凌,イグレット,9.0,2020/6/22,202044062212,タイタン賞競走,5,1:28.9,54.0,34.8,0


使わないカラム削除する。

一旦、ニューラルネットワーク用のデータを作成する。

In [54]:
horse_raw_df = horse_df.copy()
race_raw_df = race_df.copy()
race_raw_df = race_raw_df.drop(['name', 'date'], axis=1)

In [55]:
drop_columns = ['time', 'agari', 'horse_weight']

In [56]:
horse_df = horse_df.drop(drop_columns, axis=1)

In [57]:
drop_columns = ['field_condition', 'name', 'date']

In [58]:
race_df = race_df.drop(drop_columns, axis=1)

In [59]:
race_df

,clockwise,distance,field_type,place,race_id,race_round,start_time,weather
0,右,1400,ダ,大井,202044062212,12R,20:50,雨
1,右,1800,ダ,大井,202044062211,11R,20:10,雨
2,右,1200,ダ,大井,202044062210,10R,19:30,雨
3,右,1400,ダ,大井,202044062209,9R,18:55,雨
4,右,1600,ダ,大井,202044062208,8R,18:20,雨
...,...,...,...,...,...,...,...,...
27514,左,1500,ダ,川崎,201945010106,6R,13:50,晴
27515,左,900,ダ,川崎,201945010105,5R,13:20,晴
27516,左,1400,ダ,川崎,201945010104,4R,12:50,晴
27517,左,1400,ダ,川崎,201945010103,3R,12:20,晴


データを結合する。

In [60]:
train_df = horse_df.merge(race_df, on='race_id')
train_raw_df = horse_raw_df.merge(race_raw_df, on='race_id')

In [61]:
train_df.head()

,age,frame_number,horse_number,jockey,name,popular,race_date,race_id,race_name,rank,weight,win,show,clockwise,distance,field_type,place,race_round,start_time,weather
0,牡3,7.0,13,森泰斗,ワイルドホース,2.0,2020/6/22,202044062212,タイタン賞競走,1,54.0,3.7,1,右,1400,ダ,大井,12R,20:50,雨
1,牝5,6.0,11,山崎誠士,クインズオライリー,12.0,2020/6/22,202044062212,タイタン賞競走,2,54.0,81.4,1,右,1400,ダ,大井,12R,20:50,雨
2,セ6,3.0,4,西啓太,メダーリアフレイム,4.0,2020/6/22,202044062212,タイタン賞競走,3,56.0,8.6,1,右,1400,ダ,大井,12R,20:50,雨
3,牡4,8.0,14,藤本現暉,ゴールドプリンス,8.0,2020/6/22,202044062212,タイタン賞競走,4,56.0,34.4,0,右,1400,ダ,大井,12R,20:50,雨
4,牝5,4.0,7,藤田凌,イグレット,9.0,2020/6/22,202044062212,タイタン賞競走,5,54.0,34.8,0,右,1400,ダ,大井,12R,20:50,雨


In [62]:
train_df.to_csv(DATA_PATH + 'train.csv', index=False)
train_raw_df.to_csv(DATA_PATH + 'train_raw.csv', index=False)